# **Presentación del caso Forest Cover Type Prediction**

Este caso muestra un conjunto de árboles y su respectivo **tipo de cobertura**. En total se han registrado un total de **15, 120 filas o registros** y en la columna *target* se registra un total de **siete (7) categorías**:

1.   Spruce/Fir
2.   Lodgepole Pine
3.   Ponderosa Pine
4.   Cottonwood/
5.   Aspen
6.   Douglas-fir
7.   Krummholz

![Image of Yaktocat](https://miro.medium.com/max/1280/1*5IhJ18EP_XuEYkHmfazNUw.jpeg)

Debido a que la variable objetivo es una **categoría**, es decir no numérica (continua), los modelos de machine learning recomendados para predecir el precio de venta son **modelos de clasificasión**.

Sin embargo debido a que la variable objetivo **no es binaria** sino que se ha registrado un total de **7 tipos diferentes** lo recomendable es usar **modelos más complejos**:

1.   XGBoost
2.   LigthGBM

Puedes obtener más info sobre la competencia en Kaggle en el siguiente [enlace](https://www.kaggle.com/c/forest-cover-type-prediction)

# **Comprensión de los datos**

In [ ]:
"CELDA N°1"
#Importamos las librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
"CELDA N°2"
#Leemos los datos con el método read_csv estableciendo la variable Id como índice
data = pd.read_csv('https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/forest_covertype_train_preprocesado.csv')

In [ ]:
"CELDA N°3"
#Comprobamos la lectura de datos a través del método head (puedes cambiarlo con tail)
data.head()

In [ ]:
"CELDA N°4"
#Comprobamos rápidamente si hay vacíos y el tipo de variable
data.info()

# **Preprocesamiento de los Datos**

## **Verificación de vacíos**

In [ ]:
"CELDA N° 5"
#Verificación gráfica de datos perdidos con missingno
import missingno as ms
ms.matrix(data)

Como se puede apreciar en la figura **no hay presencia de valores perdidos** en ninguna columna.

## **Verificación de outliers**

In [ ]:
"CELDA N°6"
#Verificamos la existencia de outliers en las columnas numéricas
sns.boxplot(data=data)

Como se puede apreciar en la figura **no hay presencia de outliers** en ninguna columna salvo en "Horizontal_Distance_To_Roaddways". Sin embargo no es signifcativo.

## **Feature Engineering**

### *Correlación de variables*

In [ ]:
"CELDA N°7"
#Dibujamos la matriz de correlación
sns.heatmap(data.corr())

Como se puede apreciar en la figura **no hay presencia de correlación** entre las columnas.

### *Label Encoding*

In [ ]:
"CELDA N°8"
#Identificamos las columnas de tipo categórica
col_to_encoding = [col for col in data.columns if data[col].dtype == 'object']

In [ ]:
"CELDA N°9"
#Convertimos las palabras en números con la librería LabelEncoder
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
for col in col_to_encoding:
  data[col] = le.fit_transform(data[col])

In [ ]:
"CELDA N°10"
#Comprobamos el Label Encoder sobre la columna Wilderness_Area_Type
data['Wilderness_Area_Type'].unique()

### *Normalización*

In [ ]:
"CELDA N°11"
#Aplicamos MinMaxScaler para igualar la escala (0-1) para todas las columnas predictoras

X = data.drop('Cover_Type',axis=1)
y = data['Cover_Type'] #Separamos la columna que contiene al target

from sklearn.preprocessing import MinMaxScaler
data_escalada = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(data=data_escalada, columns=X.columns) #Actualizamos la data de las columnas predictoras

# **Modelamiento**

### *Partición de la data: training y test*

In [ ]:
"CELDA N°12"
#Dividimos cada variable para el entrenamiento del modelo importando la lilbrería train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.15,random_state=1) #Separa el 15% de la data para generar las predicciones

### *XGBoost*

In [ ]:
"CELDA N°13"
from xgboost import XGBClassifier #Importamos las librería XGBClassifier para generar un modelo de clasificación con el método XGBoost
xgb = XGBClassifier(learning_rate = 0.07, #indicamos un valor menor al establecido por defecto (0.1)
                    max_depth = 4, #indicamos un valor mayor al establecido por defecto (3)
                    min_child_weight = 3, #indicamos un valor mayor al establecido por defecto (1)
                    objective = 'multi:softmax', #cambiamos el objetivo por tratarse de un target con más de dos categoría,
                    num_class = 7, #especificamos el número de categorías en el target
                    eval_metric = 'mlogloss', #cambiamos la métrica por tratarse de un target con más de dos categorías
                    eval_set =[(X_test,y_test)], #referencia para cada iteración
                    n_estimators = 1000, #límites del número de iteraciones en general
                    early_stopping_rounds = 30) #indicamos un máximo de iteraciones para obtener una mejora de la métrica
xgb.fit(X_train,y_train) #entrenamos el modelo xgboost partir de X_train, y_train
predict_xgb = xgb.predict(X_test) #obtenemos las predicciones del modelo xgboost

In [ ]:
"CELDA N°14"
#Realizamos una evaluación rápida del XGBoost con la métrica accuracy score para modelos de clasificación
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predict_xgb)) #primero la data real (y_test) y luego las predicciones obtenidas del XGBoost

### *LightGBM*

In [ ]:
"CELDA N°15"
import lightgbm #Importamos las librería lightgbm para generar un modelo de clasificación con el método LightGBM
gbm = lightgbm.LGBMClassifier(learning_rate = 0.07, #indicamos un valor menor al establecido por defecto (0.1)
                    max_depth = 4, #indicamos un valor mayor al establecido por defecto (3)
                    min_child_weight = 3, #indicamos un valor mayor al establecido por defecto (1),
                    objective = 'multiclass', #cambiamos el objetivo por tratarse de un target con más de dos categorías
                    num_class = 7, #especificamos el número de categorías en el target
                    metric = 'multi_logloss', #cambiamos la métrica por tratarse de un target con más de dos categorías
                    eval_set =[(X_test,y_test)], #referencia para cada iteración
                    n_estimators = 1000) #indicamos un máximo de iteraciones en general
gbm.fit(X_train,y_train) #entrenamos el modelo lightgbm partir de X_train, y_train
predict_gbm = gbm.predict(X_test) #obtenemos las predicciones del modelo xgboost

In [ ]:
"CELDA N°16"
#Realizamos una evaluación rápida del LightGBM con la métrica accuracy score para modelos de clasificación
print(accuracy_score(y_test, predict_gbm)) #primero la data real (y_test) y luego las predicciones obtenidas del LightGBM

# **Evaluación**

## **Optimización de hiperparámetros para XGBoost**

In [ ]:
"CELDA N°17"
#Importamos la librería GridSearchCV para optimizar los parámetros numéricos del XGBoost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

#Declaramos los rangos para cada uno de los parámetros entre corchetes (lista)
parametros = {'learning_rate':[0.04, 0.06, 0.08], 'max_depth':[4,6,8], 'min_child_weight':[2,3,4]}

#Finalmente entrenamos nuevamente declarando el método de scoring (métrica)
grid_xgb = GridSearchCV(estimator = xgb,param_grid = parametros,scoring='f1-macro')
grid_xgb.fit(X_train, y_train)

In [ ]:
"CELDA N°18"
#Podemos visualizar los mejores perámetros y también la métrica obtenida
print(grid_xgb.best_params_)
print(grid_xgb.best_score_)

## **Optimización de hiperparámetros para LightGBM**

In [ ]:
"CELDA N°19"
#Finalmente entrenamos nuevamente declarando el método de scoring (métrica)
grid_gbm = GridSearchCV(estimator = gbm,param_grid = parametros,scoring='f1-macro')
grid_gbm.fit(X_train, y_train)

In [ ]:
"CELDA N°20"
#Podemos visualizar los mejores perámetros y también la métrica obtenida
print(grid_gbm.best_params_)
print(grid_gbm.best_score_)